# Qiskit Runtime REST API query & job submission

## Query API without authentication

In [16]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/programs'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")

In [17]:
[program['id'] for program in data['programs']]

['sampler', 'estimator']

## Get temporary Access token from Auth API via API Token

In [3]:
token = 'API_Token'
crn_service = 'CRN Service'

In [4]:
import requests

# with open('token') as file:
#     token=file.read()

url = 'https://iam.cloud.ibm.com/identity/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
data='grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey='+token
auth_response = requests.post(url, headers= headers, data=data)
auth_id=auth_response.json()['access_token']
auth_type = auth_response.json()['token_type']

In [18]:
# auth_response.json()

## GET available backends

In [7]:
url_backends = 'https://us-east.quantum-computing.cloud.ibm.com/backends'
headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

backends_response = requests.get(url_backends, headers=headers)

In [8]:
backends_response.json()

{'devices': ['ibm_algiers',
  'ibm_brisbane',
  'ibmq_qasm_simulator',
  'simulator_stabilizer',
  'simulator_mps',
  'simulator_extended_stabilizer',
  'simulator_statevector']}

In [9]:
backend = backends_response.json()['devices'][2]
backend

'ibmq_qasm_simulator'

## Run a job

### set up qasm circuit

In [10]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [11]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/jobs'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

job_input = {
    'program_id': 'sampler',
    "backend": backend,
    "start_session": False,
    "params": {
        "circuits": [qasm_string]
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"ck6qehjl5r7th0mrmk7g","backend":"ibmq_qasm_simulator"}


## Wait for results

In [15]:
status_result = requests.get(url+'/'+job_id,headers=headers)
print(status_result.json()['state'],job_id)

{'status': 'Completed'} ck6qehjl5r7th0mrmk7g


### Get results

In [16]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'1': 1.0}],
 'metadata': [{'shots': 1024,
   'readout_mitigation_overhead': 1.0,
   'readout_mitigation_time': 0.06403936599963345}]}

## Invalidate Token (Optional)

In [17]:
logout_url = 'https://us-east.quantum-computing.cloud.ibm.com/logout'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 
